In [1]:
# back to project root
%cd /mnt/c/research

import os
import sys
from glob import glob

import cv2
import yaml

sys.path.append("src")
from utility.logger import logger
from utility.activity_loader import load_group

/mnt/c/research


/mnt/c/research/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
room_num = "08"
surgery_num = "001"

In [9]:
cfg_path = "config/demo_config.yaml"
with open(cfg_path, "r") as f:
    cfg = yaml.safe_load(f)
with open(cfg["config_path"]["group"], "r") as f:
    grp_cfg = yaml.safe_load(f)

In [4]:
field = cv2.imread("image/field.png")

In [5]:
data_dir = os.path.join("data", room_num, surgery_num)
data_dirs = sorted(glob(os.path.join(data_dir, "*")))
for i in range(len(data_dirs)):
    if data_dirs[i].endswith("passing") or data_dirs[i].endswith("attention"):
        del data_dirs[i]
print(data_dirs)

['data/08/001/01', 'data/08/001/02', 'data/08/001/03', 'data/08/001/04', 'data/08/001/05', 'data/08/001/06', 'data/08/001/07', 'data/08/001/08', 'data/08/001/09', 'data/08/001/10', 'data/08/001/11', 'data/08/001/12', 'data/08/001/13', 'data/08/001/14', 'data/08/001/15', 'data/08/001/16', 'data/08/001/17']


In [6]:
def extract_passing(group, th_duration=10, th_interval=30):
    passing_dict = group.passing
    
    result_dict = {}
    for pair_key, passing_frame_nums in passing_dict.items():
        result_dict[pair_key] = []
        
        pre_frame_num = start_frame_num = passing_frame_nums[0]
        for frame_num in passing_frame_nums[1:]:
            if frame_num - pre_frame_num > th_interval:
                # difference between current and previous is over interval
                if pre_frame_num - start_frame_num > th_duration:
                    # append result beyond with duration
                    result_dict[pair_key].append((start_frame_num, pre_frame_num))

                start_frame_num = frame_num  # update start frame number
            
            pre_frame_num = frame_num  # update previous frame number
        else:
            # process for last frame number
            if frame_num - start_frame_num > th_duration:
                # append result beyond with duration
                result_dict[pair_key].append((start_frame_num, pre_frame_num))
                
    return result_dict

In [10]:
results = []
for data_dir in data_dirs:
    json_path = os.path.join(data_dir, ".json", "group.json")
    if os.path.exists(json_path):
        group = load_group(json_path, grp_cfg, field, logger, device="cpu")
        results.append(extract_passing(group))

2022-05-23 18:27:11,566 => load passing detector from config/passing/pass_model_lstm.yaml
2022-05-23 18:27:11 [INFO]: => load passing detector from config/passing/pass_model_lstm.yaml
2022-05-23 18:27:18,492 => load passing detector from config/passing/pass_model_lstm.yaml
2022-05-23 18:27:18 [INFO]: => load passing detector from config/passing/pass_model_lstm.yaml


KeyboardInterrupt: 